In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device_name)

block_size = 64
batch_size = 128
max_iterations = 3000
learning_rate = 3e-4
eval_iterations = 100
n_embd = 384
n_head = 4
n_layer = 4
dropout = 0.2

cuda


In [2]:
chars = ""
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))
    
# print(chars)
vocab_size = len(chars)

In [3]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

C:\Users\Yash\AppData\Local\Temp\ipykernel_4960\2481656956.py:6: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  data = torch.tensor(encode(text), dtype=torch.long)


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print('IX is', ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device_name), y.to(device_name)
    return x,y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[67,  1, 54,  ..., 73,  1, 66],
        [ 1, 73, 61,  ..., 55, 74, 60],
        [44, 61, 58,  ..., 68, 68, 64],
        ...,
        [ 3,  1, 72,  ..., 59,  0, 54],
        [61, 58,  1,  ..., 72, 58,  1],
        [ 1, 28, 68,  ..., 71, 58, 57]], device='cuda:0')
targets:
tensor([[ 1, 54, 67,  ...,  1, 66, 74],
        [73, 61, 58,  ..., 74, 60, 60],
        [61, 58,  1,  ..., 68, 64,  1],
        ...,
        [ 1, 72, 54,  ...,  0, 54, 65],
        [58,  1, 58,  ..., 58,  1, 76],
        [28, 68, 71,  ..., 58, 57,  1]], device='cuda:0')


In [5]:

x = train_data[: block_size]
y = train_data[1: block_size+1]

for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    #print('when input is', context, 'output is', target)

In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iterations)
        
        for i in range(eval_iterations):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[i] = loss.item()
            
        out[split] = losses.mean()
    model.train()
    return out

In [9]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]


class MultiHeadAttention(nn.Module):
    """ Multiple heads of self_attention in parallel """
    
    def __init__(self, n_head, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for i in range(n_head)])
        self.proj = nn.Linear(head_size * n_head, n_embd)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = torch.concat([h(x) for h in self.heads], dim = -1)
        out = self.dropout(self.proj(out))
        return out

    
    
class FeedForward(nn.Module):
    
     """ A simple linear layer followed by non-linearity """
    
     def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd)
        )
    
     def forward(self, x):
        return self.net(x)
        
        
class Block(nn.Module):
    """ Transformer block : communication followed by computation """
    
    def __init__(self, n_embd, n_head):
        # n_embd : embedding dimention, n_layer : the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x
    

class GPTLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # Final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02) 
        
        
    def forward(self, index, targets = None):
        B, T = index.shape
        
        # Initial GPT Transform steps
        # idx and targets are both (B, T) tensors of integers
        tok_emb = self.token_embedding_table(index) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device_name)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x) # (B, T, C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else :
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)         
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B,T) array of indices in current context
        for _ in range(max_new_tokens):
            # get the prediction
            logits, loss = self.forward(index)
            # (focus only on last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # ) (B, C)
            # sample from distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index
    
model = GPTLanguageModel(vocab_size)
m = model.to(device_name)
    
#context = torch.zeros((1,1), dtype=torch.long, device=device_name)
#generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
#print(generated_chars)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# PyTorch optimizer 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iterations):
    if iter % eval_iterations == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']: .4f} val loss: {losses['val']: .4f}")
        
    # sample a batch of data
    x_inputs, y_output = get_batch('train')
    
    # evaluate the loss
    logits, loss = model.forward(x_inputs, y_output)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

In [8]:
context = torch.zeros((1,1), dtype=torch.long, device=device_name)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
